In [1]:
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame
import time
import random
import math
import numpy
from pygame import gfxdraw


def main():
	pygame.init()
	game_width = 800 #1600
	game_height = 600 #900
	white = (255,255,255)
	black = (0,0,0)
	red = (255,0,0)
	green = (0,255,0)
	blue = (0,0,255)
	fps = 60
	size = 5
	mutation_rate = 0.2
	steering_weights = 0.05
	perception_radius_mutation_range = 30
	reproduction_rate = 0.000005 #0.0005
	initial_perception_radius = 100
	boundary_size = 10
	max_vel = 10
	initial_max_force = 0.02
	health = 100
	max_poison = 50
	nutrition = [20, -80]
	bots = []
	food = []
	poison = []
	oldest_ever = 0
	oldest_ever_dna = []
	gameDisplay = pygame.display.set_mode((game_width, game_height))
	clock = pygame.time.Clock()
	unique_names=[0]

	def update_damage_color():
		percent_health = bot.health/health
		lerped_colour = (max(min((1-percent_health)*255,255),0), max(min(percent_health*255,255),0), 0)
		return(lerped_colour)

	def magnitude_calc(vector):
		x = 0
		for i in vector:
			x += i**2
		magnitude = x**0.5
		return(magnitude)

	def normalise(vector):
		magnitude = magnitude_calc(vector)
		if magnitude != 0:
			vector = vector/magnitude
		return(vector)



	class create_bot(): #How to input dna????
		def __init__(self, x, y, dna=False):
			self.name=unique_names[-1]
			self.position = numpy.array([x,y], dtype='float64')
			self.velocity = numpy.array([random.uniform(-max_vel,max_vel),random.uniform(-max_vel,max_vel)], dtype='float64')
			self.acceleration = numpy.array([1, 1], dtype='float64')
			self.colour = green
			#print(magnitude_calc(self.velocity)*10)
			self.health = magnitude_calc(self.velocity)*10
			self.max_vel = 2
			self.size = self.health/20
			self.max_force = self.acceleration*1/self.size
			self.age = 1
			self.ate_counter = 0
			self.reproduction_count = 0

			unique_names.append(int(unique_names[-1])+1)

			if dna != False:
				self.dna = []
				for i in range(len(dna)):
					if random.random() < mutation_rate:
						if i < 2:
							self.dna.append(dna[i] + random.uniform(-steering_weights, steering_weights))
						else:
							self.dna.append(dna[i] + random.uniform(-perception_radius_mutation_range, perception_radius_mutation_range))

					else:
						self.dna.append(dna[i])
			else:
				self.dna = [random.uniform(-initial_max_force, initial_max_force), random.uniform(-initial_max_force, initial_max_force), random.uniform(0, initial_perception_radius), random.uniform(0, initial_perception_radius)]
			#print(self.dna)

		def update(self):
			self.velocity += self.acceleration

			self.velocity = normalise(self.velocity)*self.max_vel

			self.position += self.velocity
			self.acceleration *= 0
			#damage every update to force eating
			self.health -= 0.2
			self.colour = update_damage_color()
			self.health = min(health, self.health)
			# if self.age % 1000 == 0:
			# 	print(self.age, self.dna)
			self.age += 1

		def reproduce(self):
			for bot in bots:
				if bot.ate_counter//10>0:
					bot.ate_counter = 0
					bots.append(create_bot(self.position[0], self.position[1], self.dna))
					bot.reproduction_count += 1
			if random.random() < reproduction_rate:
				bots.append(create_bot(self.position[0], self.position[1], self.dna))


		def dead(self):
			if self.health > 0:
				return(False)
			else:
				#when dies turns to food?
				if self.position[0] < game_width - boundary_size and self.position[0] > boundary_size and self.position[1] < game_height - boundary_size and self.position[1] > boundary_size:
					food.append(self.position)
				return(True)

		def apply_force(self, force):
			self.acceleration += force
			
		def seek(self, target):
			desired_vel = numpy.add(target, -self.position)
			desired_vel = normalise(desired_vel)*self.max_vel
			steering_force = numpy.add(desired_vel, -self.velocity)
			steering_force = normalise(steering_force)*self.max_force
			return(steering_force)
			#self.apply_force(steering_force)

		def eat(self, list_of_stuff, index):
			closest = None
			closest_distance = max(game_width, game_height)
			bot_x = self.position[0]
			bot_y = self.position[1]
			item_number = len(list_of_stuff)-1

			for i in list_of_stuff[::-1]:
				item_x = i[0]
				item_y = i[1]
				distance = math.hypot(bot_x-item_x, bot_y-item_y)

				if distance < 1:
					list_of_stuff.pop(item_number)
					self.ate_counter += 1
					if index == 1:
						print(self.name,"ate poison")
					self.health += nutrition[index]

				if distance < closest_distance:
					closest_distance = distance
					closest = i
				item_number -=1
			#if it finds food to go after	
			if closest_distance < self.dna[2 + index]:
				seek = self.seek(closest) # index)
				seek *= self.dna[index]
				seek = normalise(seek)*self.max_force
				self.apply_force(seek)


		def boundaries(self):
			desired = None
			x_pos = self.position[0]
			y_pos = self.position[1]
			if x_pos-(self.size/2) < boundary_size:
				desired = numpy.array([self.max_vel, self.velocity[1]])
				steer = desired-self.velocity
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			elif x_pos+(self.size/2) > game_width - boundary_size:
				desired = numpy.array([-self.max_vel, self.velocity[1]])
				steer = desired-self.velocity
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			if y_pos-(self.size/2) < boundary_size:
				desired = numpy.array([self.velocity[0], self.max_vel])
				steer = desired-self.velocity
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			elif y_pos+(self.size/2) > game_height - boundary_size:
				desired = numpy.array([self.velocity[0], -self.max_vel])
				steer = desired-self.velocity
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)
			'''if desired != None:
				steer = desired-self.velocity
				steer = normalise(steer)*self.max_force
				self.apply_force(steer)'''


		def draw_bot(self):
			pygame.gfxdraw.aacircle(gameDisplay, int(self.position[0]), int(self.position[1]), 10, self.colour)
			pygame.gfxdraw.filled_circle(gameDisplay, int(self.position[0]), int(self.position[1]), 10, self.colour)
			pygame.draw.circle(gameDisplay, green, (int(self.position[0]), int(self.position[1])), abs(int(self.dna[2])), abs(int(min(2, self.dna[2]))))
			pygame.draw.circle(gameDisplay, blue, (int(self.position[0]), int(self.position[1])), abs(int(self.dna[3])), abs(int(min(2, self.dna[3]))))
			pygame.draw.line(gameDisplay, green, (int(self.position[0]), int(self.position[1])), (int(self.position[0] + (self.velocity[0]*self.dna[0]*25)), int(self.position[1] + (self.velocity[1]*self.dna[0]*25))), 3)
			pygame.draw.line(gameDisplay, blue, (int(self.position[0]), int(self.position[1])), (int(self.position[0] + (self.velocity[0]*self.dna[1]*25)), int(self.position[1] + (self.velocity[1]*self.dna[1]*25))), 2)
			
	for i in range(10):
		bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))
	running = True
	while(running):
		gameDisplay.fill(black)
		if len(bots)<2 or random.random() < 0.0001:
			bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))

		#random events
		if random.random()<0.1:
			unplaced = True
			while unplaced:
				rand_x = random.uniform(boundary_size, game_width-boundary_size)
				rand_y = random.uniform(boundary_size, game_height-boundary_size)

				good_position=True
				#check for bots
				for bot in bots:
					distance = math.hypot(rand_x-bot.position[0], rand_y-bot.position[1])
					if distance < 10:
						good_position = False
				#check for other foods
				for i in food[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				#check for other foods
				for i in poison[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				# if good position then..., else start over with fresh x and y	
				if good_position == True: 
					food.append(numpy.array([rand_x, rand_y], dtype='float64'))
					unplaced=False
		if random.random()<0.01:
			unplaced = True
			while unplaced:
				rand_x = random.uniform(boundary_size, game_width-boundary_size)
				rand_y = random.uniform(boundary_size, game_height-boundary_size)

				good_position=True
				#check for bots
				for bot in bots:
					distance = math.hypot(rand_x-bot.position[0], rand_y-bot.position[1])
					if distance < 20:
						good_position = False
				#check for other foods
				for i in food[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				#check for other foods
				for i in poison[::-1]:
					item_x = i[0]
					item_y = i[1]
					distance = math.hypot(rand_x-item_x, rand_y-item_y)
					if distance < 1:
						good_position = False
				# if good position then..., else start over with fresh x and y	
				if good_position == True: 
					poison.append(numpy.array([rand_x, rand_y], dtype='float64'))
					unplaced=False
		if len(poison)>max_poison:
			poison.pop(0)

		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				running = False
			#print(event)
		

		#print(bots[0].position)
		#print((bots[0].position),(bots[0].position+(-size,0)),(bots[0].position+(-size/2,size)))
		for bot in bots[::-1]:
			bot.eat(food, 0)
			bot.eat(poison, 1)
			bot.boundaries()
			#bot.seek(pygame.mouse.get_pos())
			bot.update()
			# if bot.age > oldest_ever:
			# 	oldest_ever = bot.age
			# 	oldest_ever_dna = bot.dna
			# 	#print(oldest_ever, oldest_ever_dna)
			bot.draw_bot()
			#pygame.draw.polygon(gameDisplay, bot.colour, ((bot.position),tuple(map(operator.add,bot.position,(-size,0))),tuple(map(operator.add,bot.position,(-size/2,size)))))
			if bot.dead():
				if bot.reproduction_count>2:
					print("Name:",bot.name,"died","Ate:",bot.ate_counter,"Repo:",bot.reproduction_count,"DNA:",bot.dna,"Age:",bot.age)
				else:
					print("Name:",bot.name,"died","Ate:",bot.ate_counter,"Repo:",bot.reproduction_count)
				bots.remove(bot)
			else:
				bot.reproduce()

		#if random.random()<0.02:
			#bots.append(create_bot(random.uniform(0,game_width),random.uniform(0,game_height)))

		for i in food:
			pygame.draw.circle(gameDisplay, green, (int(i[0]), int(i[1])), 3)
		#pygame.draw.circle(gameDisplay, bot.colour, (int(self.position[0]), int(self.position[1])), 10)
		for i in poison:
			pygame.draw.circle(gameDisplay, blue, (int(i[0]), int(i[1])), 3)
		pygame.display.update()
		clock.tick(fps)

	pygame.quit()
	quit()




main()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Name: 7 died Ate: 0 Repo: 0
Name: 2 died Ate: 0 Repo: 0
Name: 3 died Ate: 0 Repo: 0
Name: 9 died Ate: 0 Repo: 0
Name: 0 died Ate: 0 Repo: 0
Name: 4 died Ate: 0 Repo: 0
Name: 1 died Ate: 0 Repo: 0
Name: 5 died Ate: 0 Repo: 0
Name: 6 died Ate: 0 Repo: 0
Name: 8 died Ate: 2 Repo: 0
Name: 10 died Ate: 0 Repo: 0
Name: 11 died Ate: 0 Repo: 0
Name: 12 died Ate: 0 Repo: 0
Name: 14 died Ate: 0 Repo: 0
Name: 15 died Ate: 0 Repo: 0
16 ate poison
Name: 16 died Ate: 1 Repo: 0
Name: 19 died Ate: 0 Repo: 0
25 ate poison
Name: 25 died Ate: 8 Repo: 1
35 ate poison
Name: 35 died Ate: 1 Repo: 0
Name: 20 died Ate: 3 Repo: 1
Name: 33 died Ate: 2 Repo: 0
Name: 24 died Ate: 2 Repo: 1
41 ate poison
Name: 41 died Ate: 2 Repo: 0
Name: 32 died Ate: 4 Repo: 0
43 ate poison
Name: 43 died Ate: 6 Repo: 0
Name: 39 died Ate: 2 Repo: 0
48 ate poison
Name: 48 died Ate: 1 Repo: 0
Name: 18 died Ate: 2 Repo: 4 D

: 